---
title: Approximating compliant beams with the pseudo-rigid-body model
---

Note: this page comes with certain limitations.  First, it assumes small angle deflections, which you cannot assume for compliant mechanisms.  Second, it dos not consider displacement in x, only y and $\theta$.  Finally, it only considers transverse loads, but not the impact of axial loading.

With that said, it's an interesting and simple derivation.  For more information about large displacements, I encourage you to look at:

Howell, L. L., and A. Midha. “Parametric Deflection Approximations for End-Loaded, Large-Deflection Beams in Compliant Mechanisms.” Journal of Mechanical Design 117, no. 1 (March 1, 1995): 156–65. https://doi.org/10.1115/1.2826101.


In [1]:
import sympy

In [2]:
q = sympy.Symbol('q')
d = sympy.Symbol('d')
L = sympy.Symbol('L')
P = sympy.Symbol('P')
h = sympy.Symbol('h')
b = sympy.Symbol('b')
E = sympy.Symbol('E')
x = sympy.Symbol('x')

In [3]:
subs = {}
#subs[k]=1000
subs[P]=.1
subs[L]=.1
subs[b]=.01
subs[h]=.01
subs[E]=1e7
subs[x]=.5

![](/figures-external/beam-deflection.jpg)

## Approximating a cantilever with a single revolute joint

Arbitrarily placing a compliant joint halfway along its length can be used to approximate a cantilever beam.

## Euler-Bernoulli Equations

Point Load at the end:

$$d = \frac{PL^3}{3EI}$$

Cross-sectional moment of inertia for a rectangle

$$I = \frac{bh^3}{12}$$

Inserted:
$$d = \frac{4PL^3h^3}{Eb}$$

$$E_{rectangle} = \frac{4PL^3h^3}{db}$$
$$E = \frac{PL^3}{3dI}$$

Cross sectional moment of inertia for a rectangle

In [4]:
I = b*h**3/12

In [5]:
d1 = P*L**3/3/E/I
d1.subs(subs)

0.00400000000000000

In [6]:
q1 = P*L**2/2/E/I

In [7]:
q1.subs(subs)

0.0600000000000000

## 2-Link Approximation

### Matching Displacement
x at .5

$$d = L(1-x) \sin{\theta}$$

<!--$$\sin^{-1}\frac{d}{L}=\theta$$-->

$$\tau=k\theta = PL(1-x)\cos\theta $$

Using a small Angle approximation, $\cos\theta = 1$

$$k\theta = PL(1-x)$$
$$\theta = \frac{PL(1-x)}{k}$$
$$d=L(1-x)\sin\left(\frac{PL(1-x)}{k}\right)$$

$$d=L(1-x)\sin\left(\frac{PL(1-x)}{k}\right)=\frac{PL^3}{3EI}$$
$$(1-x)\sin\left(\frac{PL(1-x)}{k}\right)=\frac{PL^2}{3EI}$$

$$\frac{PL(1-x)}{k}=\sin^{-1} \left(\frac{PL^2}{3EI(1-x)}\right)$$

$${k}=\frac{PL(1-x)}{\sin^{-1} \left(\frac{PL^2}{3EI(1-x)}\right)}$$

In [8]:
k1 = P*L*(1-x)/(sympy.asin(P*L**2/(3*E*I*(1-x))))
k1.subs(subs)

0.0624332120451467

The displacement matches

In [9]:
d2 = L*(1-x)*sympy.sin(P*L*(1-x)/k1)
d2.subs(subs)

0.00400000000000000

But the orientation does not

In [10]:
q2 = P*L*(1-x)/k1

In [11]:
q2.subs(subs)

0.0800855800336590

### Matching Theta

From Cantilever beam equations:

$$\theta = \frac{PL^2}{2EI}$$

From approximation above, again assuming small angles:

$$\theta = \frac{PL(1-x)}{k}$$

$$\theta = \frac{PL^2}{2EI}= \frac{PL(1-x)}{k}$$
$$\frac{L}{2EI}= \frac{1-x}{k}$$
$$k=\frac{2EI(1-x)}{L}$$

Using the value solved for to equate orientation:

In [12]:
k2 = 2*E*I*(1-x)/(L)

Now orientation matches

In [13]:
q3 = P*L*(1-x)/k2

In [14]:
q3.subs(subs)

0.0600000000000000

But displacement does not

In [15]:
d3 = L*(1-x)*sympy.sin(P*L*(1-x)/k2)


In [16]:
d1.subs(subs)

0.00400000000000000

In [17]:
d3.subs(subs)

0.00299820032397223

### Matching Both
Now asking the question, what location x permits you to accurately model the deflection and angle of a cantilever beam with a single joint?

In [18]:
del subs[x]

Create an error vector

In [19]:
error = []
error.append(d1-d2)
error.append(q1-q2)
error= sympy.Matrix(error)
error = error.subs(subs)
error

Matrix([
[                        0],
[0.06 - asin(0.04/(1 - x))]])

import optimization toolkit

In [20]:
import scipy.optimize

Turn "error" into a function that can be run using the ```sympy.lambdify``` function

In [21]:
f = sympy.lambdify((x),error)

Scipy.optimize.minimize needs args supplied as a list, so define a new wrapper function that formats inputs correctly

In [22]:
def f2(args):
    a = f(*args)
    b = (a**2).sum()
    return b

In [23]:
sol = scipy.optimize.minimize(f2,[.25])
sol

      fun: 2.0974856866229214e-09
 hess_inv: array([[65.11062045]])
      jac: array([-8.23593665e-06])
  message: 'Optimization terminated successfully.'
     nfev: 14
      nit: 4
     njev: 7
   status: 0
  success: True
        x: array([0.33242421])

Now add x back to the list of substitutions

In [24]:
subs[x]=sol.x[0]

So a virtual joint at x=1/3 correctly approximates displacement and orientation.

In [25]:
d2.subs(subs)

0.00400000000000000

In [26]:
q2.subs(subs)

0.0599542016846748